In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import pymongo
import datetime as dt
import csv
from pandas import DataFrame


In [2]:
# initialize chromedriver connection
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# define website pages for games scrapping
url_list = "https://www.microsoft.com/en-us/store/most-played/games/xbox"

In [4]:
# get links to top 50 played US games pages from the most play site (US)

    url = url_list 
    web_page_links=[]
    
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html)
    data = soup.findAll('div', attrs={'class':'m-channel-placement-item'})
    for div in data:
        links = div.findAll('a')
        for a in links:
            web_page_links.append(a['href'])
    print(len(web_page_links))
    
       

50


In [5]:
# scrape game page - build URL
base_url = "https://www.microsoft.com/"



In [6]:
# Define list variables to store scraped data
game_detail_name=[]
game_page_links = []
game_genre = []
game_rating_text =[]
game_rating = []
rating_detail = []
game_descrip = []
screen_shot_links =[]
trailer_links = []
date_string=[]

# need to convert timestamp into string for mongodb storage
date_stamp = str(dt.date.today())

for i in range(len(web_page_links)):

#     print(web_page_links)

# temp variables
    rat_text = []
    genre_text = []
    scr_links = []

# scrape game genre
    url = base_url+web_page_links[i]
    print(url)

    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html)
    date_string.append(date_stamp)
    
# find game name
    data = soup.find('h1')
#     print(data.text)
    game_detail_name.append(data.text)
#     print(game_detail_name)


# find game genres    
    data = soup.findAll('div', attrs={'class': 'buybox-metadata'})
    for div in data:
        links = div.findAll('a')
        for a in links:
            genre_text.append(a.string)
#             print(a.string)
    game_genre.append(genre_text)
#     print(game_genre)

            

# find game rating text 
    data = soup.findAll('div', attrs={'class': 'c-paragraph'})
#     print(data)
    for row in data:
        rat_text.append(row.text)
#         print(row.text)  
    game_rating_text.append(rat_text)
#     print(game_rating_text)

# find game rating 
    data = soup.find('a', attrs={'class': 'c-hyperlink c-label'})
    game_rating.append(data.attrs['aria-label'])
#     print(game_rating)

# find game descript
    data = soup.find('p').text
    game_descrip.append(data)
#     print(game_descrip)

#find game screenshots
    data = soup.findAll('img', attrs={'class': 'f-screenshot-fixed-size'})
    for row in data:
        screen = row.attrs['data-srcset']
        scr_links.append(screen)
    screen_shot_links.append(scr_links)
#     print(screen_shot_links)


            


https://www.microsoft.com//en-us/p/fortnite/bt5p2x999vh2?cid=msft_web_chart
https://www.microsoft.com//en-us/p/call-of-duty-modern-warfare/c5dtj99626k3?cid=msft_web_chart
https://www.microsoft.com//en-us/p/grand-theft-auto-v/bpj686w6s0nh?cid=msft_web_chart
https://www.microsoft.com//en-us/p/rocket-league/c125w9bg2k0v?cid=msft_web_chart
https://www.microsoft.com//en-us/p/tom-clancys-rainbow-six-siege/c12t09dsvp8j?cid=msft_web_chart
https://www.microsoft.com//en-us/p/minecraft/9nblggh537bl?cid=msft_web_chart
https://www.microsoft.com//en-us/p/nba-2k21/9p0q1f7p0kr1?cid=msft_web_chart
https://www.microsoft.com//en-us/p/apex-legends/bv9ml45j2q5v?cid=msft_web_chart
https://www.microsoft.com//en-us/p/roblox/bq1tn1t79v9k?cid=msft_web_chart
https://www.microsoft.com//en-us/p/madden-nfl-21-xbox-one/9pblvtgxlv7d?cid=msft_web_chart
https://www.microsoft.com//en-us/p/destiny-2/bpq955fqfph6?cid=msft_web_chart
https://www.microsoft.com//en-us/p/cyberpunk-2077/bx3m8l83bbrw?cid=msft_web_chart
https://w

In [7]:
print("name:", len(game_detail_name), game_detail_name)
print("rating", len(game_rating))
print("genre:", len(game_genre))
print("rating text", len(game_rating_text))
print("description", len(game_descrip))
print("screenshot links", len(screen_shot_links))




name: 50 ['Fortnite', 'Call of Duty®: Modern Warfare®', 'Grand Theft Auto V', 'Rocket League®', "Tom Clancy's Rainbow Six Siege", 'Minecraft', 'NBA 2K21', 'Apex Legends™', 'ROBLOX', 'Madden NFL 21 Xbox One', 'Destiny 2', 'Cyberpunk 2077', 'ARK: Survival Evolved', 'NBA 2K21 Next Generation', 'Red Dead Redemption 2', 'Forza Horizon 4', 'Call of Duty®: Black Ops Cold War - Xbox Series X|S', 'Overwatch® Origins Edition', 'Dead by Daylight: Special Edition', 'The Elder Scrolls V: Skyrim Special Edition', 'STAR WARS™ Battlefront™ II', 'NBA 2K20', 'Madden NFL 20', 'FIFA 21 Xbox One', "PLAYERUNKNOWN'S BATTLEGROUNDS", 'Sea of Thieves', 'Fallout 76', 'SMITE', 'FOR HONOR™ Standard Edition', 'Halo: The Master Chief Collection', 'Gears 5', 'STAR WARS Jedi: Fallen Order™', 'The Elder Scrolls Online: Tamriel Unlimited', 'Rec Room', 'Mortal Kombat 11', 'The Sims™ 4', 'Call of Duty: Black Ops III', 'Battlefield™ V', 'Madden NFL 21 Xbox Series X|S', 'Fallout 4', 'Halo 5: Guardians', 'Call of Duty®: Blac

In [8]:
# getting data together (first zip to a tuple, then covert to list of tuples)

game_detail = zip(date_string, game_detail_name, game_descrip, game_genre, game_rating, game_rating_text, screen_shot_links)
print(game_detail)

In [9]:
# convert to list of tuples
list_game_detail = list(game_detail)
# print(list_game_detail)

In [10]:
# load data into pandas dataframe with columns

detail_df = DataFrame(list_game_detail, columns=['Date', 'Game', 'Description', 'Genre', 'Rating', 'Rating Notes', 'Screen Links'])
# print(len(detail_df))
detail_df

,Date,Game,Description,Genre,Rating,Rating Notes,Screen Links
0,2021-01-04,Fortnite,The hunt is on in Fortnite Chapter 2 - Season ...,"[‪Action & adventure‬, ‪Other‬, ‪Shooter‬, ‪Si...","Rating Board ESRB, TEEN. Rated For ages 13 and...","[Violence, \nUsers Interact\nIn-Game Purchases...",[//store-images.s-microsoft.com/image/apps.296...
1,2021-01-04,Call of Duty®: Modern Warfare®,Experience the ultimate online playground with...,[‪Shooter‬],"Rating Board ESRB, MATURE 17+. Rated For ages ...","[Blood and Gore, Suggestive Themes, Use of Dru...",[//store-images.s-microsoft.com/image/apps.116...
2,2021-01-04,Grand Theft Auto V,"When a young street hustler, a retired bank ro...",[‪Action & adventure‬],"Rating Board ESRB, MATURE 17+. Rated For ages ...","[Nudity, Use of Alcohol, Blood and Gore, Stron...",[//store-images.s-microsoft.com/image/apps.249...
3,2021-01-04,Rocket League®,PLAY ROCKET LEAGUE FOR FREE!\n\nDownload and c...,[‪Sports‬],"Rating Board ESRB, EVERYONE. Rated For all ages.",[\nUsers Interact\nIn-Game Purchases\n],[//store-images.s-microsoft.com/image/apps.475...
4,2021-01-04,Tom Clancy's Rainbow Six Siege,Squad up and breach in to explosive 5v5 PVP ac...,[‪Shooter‬],"Rating Board ESRB, MATURE 17+. Rated For ages ...","[Blood, Drug Reference, Strong Language, Viole...",[//store-images.s-microsoft.com/image/apps.298...
5,2021-01-04,Minecraft,Explore randomly generated worlds and build am...,"[‪Action & adventure‬, ‪Family & kids‬]","Rating Board ESRB, EVERYONE 10+. Rated For age...","[Fantasy Violence, \nOnline Interactions Not R...",[//store-images.s-microsoft.com/image/apps.591...
6,2021-01-04,NBA 2K21,NBA 2K21 is the latest title in the world-reno...,[‪Sports‬],"Rating Board ESRB, EVERYONE. Rated For all ages.",[\nUsers Interact\nIn-Game Purchases (Includes...,[//store-images.s-microsoft.com/image/apps.132...
7,2021-01-04,Apex Legends™,"Conquer with character in Apex Legends, a free...",[‪Shooter‬],"Rating Board ESRB, TEEN. Rated For ages 13 and...","[Blood, Violence, \nUsers Interact\nIn-Game Pu...",[//store-images.s-microsoft.com/image/apps.643...
8,2021-01-04,ROBLOX,Roblox is the ultimate virtual universe that l...,"[‪Action & adventure‬, ‪Family & kids‬, ‪Other...","Rating Board ESRB, EVERYONE 10+. Rated For age...","[Fantasy Violence, \nUsers Interact\nIn-Game P...",[//store-images.s-microsoft.com/image/apps.562...
9,2021-01-04,Madden NFL 21 Xbox One,This game has a free trial,[‪Sports‬],"Rating Board ESRB, EVERYONE. Rated For all ages.","[Mild Lyrics, \nUsers Interact\nIn-Game Purcha...",[//store-images.s-microsoft.com/image/apps.265...


In [11]:
#export pandas dataframe to cvs file, appending to prior data 
detail_df.to_csv('games_detail_db.csv', header=True)

In [12]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [13]:
# drop existing db to prevent duplicate records
db.ms_games_detail.drop()

NameError: name 'db' is not defined

In [14]:
# Define database and collection
db = client.games_db
collection = db.ms_games_detail

In [15]:
# Dictionary to be inserted as a MongoDB document
data_dict = detail_df.to_dict("records")

# print(data_dict)

In [16]:
# Insert collection
for row in data_dict:
    collection.update_many(row, {"$set": row}, upsert=True)

In [17]:
games = db.ms_games_detail.find()

for game in games:
    print(game)

{'_id': ObjectId('5fecc8f8b07a38386c3c09b4'), 'Date': '2020-12-30', 'Description': 'The hunt is on in Fortnite Chapter 2 - Season 5: Zero Point. Join Agent Jones as he enlists the greatest Hunters across Realities like The Mandalorian to stop others from escaping the loop. Battle for honor in an ancient arena, take on bounties from new characters, and try out new Exotic weapons that pack a punch. Your targets are waiting. Join the Hunt.\n\nFortnite is the completely free multiplayer game where you and your friends can jump into Battle Royale or Fortnite Creative. Download now for FREE and jump into the action.', 'Game': 'Fortnite', 'Genre': ['\u202aAction & adventure\u202c', '\u202aOther\u202c', '\u202aShooter\u202c', '\u202aSimulation\u202c', '\u202aStrategy\u202c'], 'Rating': 'Rating Board ESRB, TEEN. Rated For ages 13 and up.', 'Rating Notes': ['Violence', '\nUsers Interact\nIn-Game Purchases\n', '\nOnline Interactions Not Rated by the ESRB\n'], 'Screen Links': ['//store-images.s-mi